# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.5014967773762162, 4.938092403135365, 5.6902254857320305, 3.564911957077405, 4.293464136372024, 6.993784929302416, 5.719132202192078, 5.405541833462546, 5.634111572978425, 5.197741017417594]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[6.847674025235913, 7.164902497643237, 6.998113575519019, 7.640472435118546, 8.76884941581002, 9.2406576257458, 9.000704214905308, 7.358303918509394, 6.57135281712176, 7.415302213478479]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.197741017417594, 5.634111572978425, 7.415302213478479, 5.405541833462546, 6.57135281712176, 7.358303918509394, 5.719132202192078, 9.000704214905308, 9.2406576257458, 8.76884941581002]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. Means of the two Gaussian distributions: 5 and 8
2. Variances of the two Gaussian distributions: 1 and 1
3. Weights of the two distributions (which is uncertain due to the coin flip): Depends on the actual number of samples selected from each distribution.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1) / len(s1), sum(s2) / len(s2)]
var = [sum(map(lambda x: (x - mean[0])**2, s1)) / len(s1), sum(map(lambda x: (x - mean[1])**2, s2)) / len(s2)]


print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.415302213478479, 7.358303918509394, 9.000704214905308, 9.2406576257458, 8.76884941581002]
[5.197741017417594, 5.634111572978425, 5.405541833462546, 6.57135281712176, 5.719132202192078]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 8.356763477689801,  mean_2 = 5.70557588863448
var_1 = 0.6498028323477654,  var_2 = 0.22055546052742292


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)


probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.197741017417594
probability of observing that point if it came from cluster 0 =  4.529096069387172e-06
probability of observing that point if it came from cluster 1 =  0.12769030976283052
point =  5.634111572978425
probability of observing that point if it came from cluster 0 =  9.460874747984668e-05
probability of observing that point if it came from cluster 1 =  1.7163040585197373
point =  7.415302213478479
probability of observing that point if it came from cluster 0 =  0.21493427838929824
probability of observing that point if it came from cluster 1 =  1.616415172439085e-13
point =  5.405541833462546
probability of observing that point if it came from cluster 0 =  2.036981527556622e-05
probability of observing that point if it came from cluster 1 =  0.7170445203059893
point =  6.57135281712176
probability of observing that point if it came from cluster 0 =  0.014086230185395215
probability of observing that point if it came from cluster 1 =  0.0008153714974289301
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([(x - mean[0])**2 * prob for x, prob in zip(data, prob_s0_x)]) / sum(prob_s0_x), sum([(x - mean[1])**2 * prob for x, prob in zip(data, prob_s1_x)]) / sum(prob_s1_x)]


print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 8.072800205538872,  mean_2 = 5.503732682828927
var_1 = 0.9729227230320431,  var_2 = 0.056486857630339234


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_s0_x = []  # Updated P(S_0 | X_i)
prob_s1_x = []  # Updated P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        
    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Updated Probability of coming from S_1 = " + str(p[1]))
    print("Updated Probability of coming from S_2 = " + str(p[2]))
    print()


5.197741017417594
Updated Probability of coming from S_1 = 0.011435099387608769
Updated Probability of coming from S_2 = 0.9885649006123912

5.634111572978425
Updated Probability of coming from S_1 = 0.018960815716219374
Updated Probability of coming from S_2 = 0.9810391842837807

7.415302213478479
Updated Probability of coming from S_1 = 0.9999999999999682
Updated Probability of coming from S_2 = 3.17094149746939e-14

5.405541833462546
Updated Probability of coming from S_1 = 0.009842699540707092
Updated Probability of coming from S_2 = 0.9901573004592928

6.57135281712176
Updated Probability of coming from S_1 = 0.999625782307995
Updated Probability of coming from S_2 = 0.00037421769200498206

7.358303918509394
Updated Probability of coming from S_1 = 0.9999999999997793
Updated Probability of coming from S_2 = 2.2072606146579682e-13

5.719132202192078
Updated Probability of coming from S_1 = 0.029984804346060566
Updated Probability of coming from S_2 = 0.9700151956539393

9.000704214

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
hard_assignments = [0 if prob0 > prob1 else 1 for prob0, prob1 in zip(prob_s0_x, prob_s1_x)]
